In [ ]:
# !pip install requests pandas

In [ ]:
import requests
import json
from csv import DictReader, excel_tab
from io import StringIO

import pandas as pd

In [ ]:
uuids = ['a1d17fdd270a69c813b872a927dfa5f3']

search_api = 'https://search.api.hubmapconsortium.org/v3/portal/search'

In [ ]:
## from: https://github.com/hubmapconsortium/user-templates-api/blob/development/src/user_templates_api/templates/jupyter_lab/templates/celltypes_salmon/template.txt
def get_uuid_to_hubmap(uuids): 
    '''
    Retrieve a dictionary mapping uuids to hubmap ids.

    Parameters
    ----------
    uuids : list
        list with uuids
    
    Returns
    -------
    dictionary mapping uuids to hubmap ids
    '''
    ## Fetch metadata, and read it into a dataframe
    response = requests.post(
        'https://portal.hubmapconsortium.org/metadata/v0/datasets.tsv', json={'uuids': uuids}
    )
    metadata = list(DictReader(StringIO(response.text), dialect=excel_tab))
    metadata = pd.DataFrame(metadata[1:])

    ## Create mapping from uuid to hubmap id
    uuid_to_hubmap = dict(zip(metadata['uuid'], metadata['hubmap_id']))
    return uuid_to_hubmap

uuid_to_hubmap = get_uuid_to_hubmap(uuids)
uuid_to_hubmap

In [ ]:
## adapted from: https://github.com/hubmapconsortium/user-templates-api/blob/development/src/user_templates_api/templates/jupyter_lab/templates/api_tutorial/template.txt
# The Search API supports Elasticsearch queries:

hits = json.loads(
    requests.post(
        search_api,
        json={
            "size": 10000,  # To make sure the list is not truncted, set this high.
            "query": {"ids": {"values": uuids}},
            "_source": ["files"],
        },  # Documents are large, so only request the fields we need.
    ).text
)["hits"]["hits"]

hits = [h for h in hits if len(h['_source']['files']) > 0]

In [ ]:
hits